## RAG_Clova

#### Env

Python Version : Python 3.12.2

requirements : 
https://ssl.pstatic.net/static/clova/service/hyperclova/cookbook/rag/requirements.txt

In [1]:
# Vector DB인 Milvus와 관련된 모듈들은 모듈의 용도를 명확히 구분하기 위해 Vector DB 구축 단계에서 불러왔으며 해당 부분에서 코드를 확인하실 수 있습니다.
import json
import os
import subprocess
from langchain_community.document_loaders import UnstructuredHTMLLoader
from pathlib import Path
import base64
import http.client
from tqdm import tqdm
import requests
import dotenv
dotenv.load_dotenv()
CLOVA_api_key=os.getenv("X-NCP-CLOVASTUDIO-API-KEY")
NCP_API_api_key=os.getenv("X-NCP-APIGW-API-KEY")
clova_REQUEST_ID=os.getenv("X-NCP-CLOVASTUDIO-REQUEST-ID")

#### 1. Raw Data → Connecting

In [2]:
url_to_filename_map = {}
wget_path="/opt/homebrew/bin/wget"
with open("clovastudiourl.txt", "r") as file:
    urls = [url.strip() for url in file.readlines()]
 
folder_path = "clovastudioguide"
 
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for url in urls:
    filename = url.split("/")[-1] + ".html"
    file_path = os.path.join(folder_path, filename)
    subprocess.run([wget_path, "-O", file_path, url], check=True)
    url_to_filename_map[url] = filename
 
with open("url_to_filename_map.json", "w") as map_file:
    json.dump(url_to_filename_map, map_file)

--2024-08-22 19:59:05--  https://guide.ncloud-docs.com/docs/clovastudio-info
Resolving guide.ncloud-docs.com (guide.ncloud-docs.com)... 104.18.6.159, 104.18.7.159
Connecting to guide.ncloud-docs.com (guide.ncloud-docs.com)|104.18.6.159|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-08-22 19:59:05 ERROR 403: Forbidden.



CalledProcessError: Command '['/opt/homebrew/bin/wget', '-O', 'clovastudioguide/clovastudio-info.html', 'https://guide.ncloud-docs.com/docs/clovastudio-info']' returned non-zero exit status 8.

In [2]:
import os
import json
import requests
from tqdm import tqdm

url_to_filename_map = {}
folder_path = "clovastudioguide"

with open("clovastudiourl.txt", "r") as file:
    urls = [url.strip() for url in file.readlines()]

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

for url in tqdm(urls, desc="Downloading"):
    filename = url.split("/")[-1] + ".html"
    file_path = os.path.join(folder_path, filename)
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        with open(file_path, "wb") as file:
            file.write(response.content)
        
        url_to_filename_map[url] = filename

    except requests.RequestException as e:
        print(f"An error occurred while downloading {url}: {e}")

with open("url_to_filename_map.json", "w") as map_file:
    json.dump(url_to_filename_map, map_file, indent=4)

print("Download complete. Mapping saved to url_to_filename_map.json")


Downloading: 100%|██████████| 7/7 [00:04<00:00,  1.48it/s]

Download complete. Mapping saved to url_to_filename_map.json


##### Quote

txt 파일을 작성할 때는 한 줄에 하나의 URL을 작성해야 합니다. 이때, 각 URL은 줄바꿈으로 구분되어야 합니다.

그리고 html 데이터를 로딩하기 전, robots.txt를 통해 데이터로 활용할 사이트의 접근 허용 여부를 확인해야합니다. NCP의 클로바스튜디오 사용 가이드의 경우 User-agent: * Allow:/ 로 접근이 가능합니다.

예제에서 활용한 HTML은, 네이버클라우드 플랫폼(NCP)의 클로바스튜디오 사용 가이드중 CLOVA Studio("AI Services" → "CLOVA Studio")와 관련된 모든 안내 페이지를 데이터로 활용했습니다.

LangChain을 활용해 로딩한 html은, 파일을 저장한 디렉토리의 주소를 metadata의 'source'로 가져오게 됩니다. 추후 답변 제공시, 디렉토리 주소가 아닌 실제 URL을 제공하기 위해, LangChain이 로딩한 데이터를 수정해야합니다.

html을 로딩할 때 파일명을 URL로 할 경우, /와 : 기호로 인해 파일명이 깨지게 됩니다. 따라서 원본 URL과 로딩한 HTML 파일을 쌍으로 mapping하고, 이 정보를 json 형식으로 "url_to_filename_map"에 저장합니다.

이후, html 파일의 저장 경로가 담긴 'source'를, 이 json 파일과 연결해 실제 URL로 변환해줍니다. 위 코드는, json 파일을 저장하는 단계까지입니다.

##### LangChain 활용 HTML 로딩



In [3]:
# 폴더 이름에 맞게 수정
html_files_dir = Path('/Users/minu/dev/Naver/enjoyAll-AI/clovastudioguide')
 
html_files = list(html_files_dir.glob("*.html"))
 
clovastudiodatas = []
 
for html_file in html_files:
    loader = UnstructuredHTMLLoader(str(html_file))
    document_data = loader.load()
    clovastudiodatas.append(document_data)
    print(f"Processed {html_file}")

Processed /Users/minu/dev/Naver/enjoyAll-AI/clovastudioguide/clovastudio-info.html
Processed /Users/minu/dev/Naver/enjoyAll-AI/clovastudioguide/clovastudio-glossary.html
Processed /Users/minu/dev/Naver/enjoyAll-AI/clovastudioguide/clovastudio-screen.html
Processed /Users/minu/dev/Naver/enjoyAll-AI/clovastudioguide/clovastudio-playground01.html
Processed /Users/minu/dev/Naver/enjoyAll-AI/clovastudioguide/clovastudio-playground.html
Processed /Users/minu/dev/Naver/enjoyAll-AI/clovastudioguide/clovastudio-start.html
Processed /Users/minu/dev/Naver/enjoyAll-AI/clovastudioguide/clovastudio-procedure.html


##### Quote

각 HTML에는 page_content에 모든 텍스트가, metadata의 'source'에는 저장 경로가 기록되어 있습니다. 위는, 로딩된 데이터 중 하나인 clovastudio-info.html의 형태입니다. metadata의 'source'에 실제 URL이 아닌 html 파일이 저장된 디렉토리 경로가 담긴 것을 확인할 수 있습니다. Mapping을 통해 이 'source'를 실제 URL로 바꾸는 작업을 다음 단계에 진행하게 됩니다.

##### Mapping 정보를 활용해 'source'를 실제 URL로 대체

In [5]:
with open("url_to_filename_map.json", "r") as map_file:
    url_to_filename_map = json.load(map_file)
 
filename_to_url_map = {v: k for k, v in url_to_filename_map.items()}
 
# langchainwikidatas 리스트의 각 Document 객체의 'source' 수정
for doc_list in clovastudiodatas:
    for doc in doc_list:
        extracted_filename = doc.metadata["source"].split("/")[-1]
        if extracted_filename in filename_to_url_map:
            doc.metadata["source"] = filename_to_url_map[extracted_filename]
        else:
            print(f"Warning: {extracted_filename}에 해당하는 URL을 찾을 수 없습니다.")

In [6]:
# 이중 리스트를 풀어서 하나의 리스트로 만드는 작업
clovastudiodatas_flattened = [item for sublist in clovastudiodatas for item in sublist]

In [7]:
clovastudiodatas_flattened 

[Document(page_content="Login\n\nKorean\n\nEnglish\n\nJa - 日本語\n\nKorean\n\nAPI 가이드\n\nCLI 가이드\n\n내용 x\n\nHOME\n\n포털 및 콘솔\n\n네이버 클라우드 플랫폼 사용 환경\n\nCompute\n\nContainers\n\nStorage\n\nNetworking\n\nDatabase\n\nSecurity\n\nAI Services\n\nAI·NAVER API\n\nApplication Services\n\nBig Data & Analytics\n\nBlockchain\n\nBusiness Applications\n\nContent Delivery\n\nDeveloper Tools\n\nDigital Twin\n\nGaming\n\nHybrid & Private Cloud\n\nInternet of Things\n\nManagement & Governance\n\nMedia\n\nMigration\n\nCLOVA Studio 개념\n\n콘텐츠 공유\n\n인쇄\n\n공유\n\nPDF\n\n내용\n\nCLOVA Studio 개념\n\n인쇄\n\n공유\n\nPDF\n\n기사 요약\n\n이 요약이 도움이 되었나요?\n\n의견을 보내 주셔서 감사합니다.\n\nClassic/VPC 환경에서 이용 가능합니다.\n\nCLOVA Studio를 이용하는 전체 시나리오를 학습하기에 앞서 CLOVA Studio에 대한 몇 가지 개념을 설명합니다.\n\n프롬프트와 결괏값\n\n프롬프트는 CLOVA Studio에서 작업을 수행하기 위해 입력해야 할 내용을 의미합니다. CLOVA Studio에서 입력한 프롬프트를 기반으로 HyperCLOVA 언어 모델이 결괏값을 생성합니다. HyperCLOVA 언어 모델은 확률을 기반으로 작동하기 때문에 같은 프롬프트를 입력하더라도 다른 결괏값이 생성될 수 있습니다. <예시> 프롬프트에 '원숭이 엉덩이는 빨개'를 입력한 경우, 높은 확률로 '빨간 건 사과, 사과는 맛있어'

#### 2. Chunking



임베딩 모델이 처리할 수 있는 적당한 크기로 raw data를 나누는 것은 매우 중요합니다. 이는 임베딩 모델마다 한 번에 처리할 수 있는 토큰 수의 한계가 있기 때문입니다. CLOVA Studio의 문단 나누기 API는 모델이 직접 문장들간의 의미 유사도를 찾아 최적의 chunk 개수와 사용자가 원하는 1개 chunk의 크기(글자 수)를 직접 설정하여 문단을 나눌 수도 있습니다. 추가로, 후처리(postProcess = True)를 통해 chunk당 글자 수의 상한선과 하한선을 postProcessMaxSize와 postProcessMinSize로 조절할 수도 있습니다.

In [20]:
class SegmentationExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id
 
    def _send_request(self, completion_request):
        headers = {
            "Content-Type": "application/json; charset=utf-8",
            "X-NCP-CLOVASTUDIO-API-KEY": self._api_key,
            "X-NCP-APIGW-API-KEY": self._api_key_primary_val,
            "X-NCP-CLOVASTUDIO-REQUEST-ID": self._request_id
        }
 
        conn = http.client.HTTPSConnection(self._host)
        print("ht2")
        conn.request(
            "POST",
            "/testapp/v1/api-tools/segmentation/RAG_langchainwiki_test", # If using Service App, change 'testapp' to 'serviceapp', and corresponding app id.
            json.dumps(completion_request),
            headers
        )
        print("ht2")
        response = conn.getresponse()
        print(response.length)
        result = json.loads(response.read().decode(encoding="utf-8"))
        conn.close()
        return result
 
    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res["status"]["code"] == "20000":
            return res["result"]["topicSeg"]
        else:
            raise ValueError(f"{res}")
 
 
if __name__ == "__main__":
    segmentation_executor = SegmentationExecutor(
        host="clovastudio.apigw.ntruss.com",
        api_key=CLOVA_api_key,
        api_key_primary_val=NCP_API_api_key,
        request_id=clova_REQUEST_ID
    )
 
    chunked_html = []
 
    for htmldata in tqdm( clovastudiodatas_flattened):
        try:
            request_data = {
                "postProcessMaxSize": 100,
                "alpha": 1.5,
                "segCnt": -1,
                "postProcessMinSize": 0,
                "text": htmldata.page_content,
                "postProcess": False
            }

            request_json_string = json.dumps(request_data)
            request_data = json.loads(request_json_string, strict=False)
            print(1)
            response_data = segmentation_executor.execute(request_data)
        except Exception as e:
            print(f"Error occurred. Message: {e}")
        
        for paragraph in response_data:
            chunked_document = {
                "source": htmldata.metadata["source"],
                "text": paragraph
            }
            chunked_html.append(chunked_document)
 
print(len(chunked_html))

  0%|          | 0/7 [00:00<?, ?it/s]

1
ht2
ht2
None
Error occurred. Message: {'status': {'code': '40000', 'message': ''}, 'result': None}


NameError: name 'response_data' is not defined